In [113]:
import os
import torch  
import torch.nn as nn 
import torch.optim as optim 
from torchvision import datasets, models, transforms 
from torch.utils.data import dataloader, DataLoader, random_split 
from PIL import Image 

In [102]:
data_dir = "/kaggle/input/plantdisease/PlantVillage"
 
batch_size = 64 
image_size = 224

train_transforms = transforms.Compose([
    transforms.Resize((224,224)), #insure images fits expected input shape
    transforms.RandomHorizontalFlip(), #Add Augmentation
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(), #covert to pytorch format 
    transforms.Normalize([0.485,0.486,0.406],  # Standard RGB means for models 
                         [0.229,0.224,0.255])                       
])

In [104]:
#Load Dataset 
full_dataset = datasets.ImageFolder(root=data_dir, transform=train_transforms)
# Split: 80% train, 20% test
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2, pin_memory=True)


#num of classes 
num_classes = len(full_dataset.classes)
print(f"Detected classes: {full_dataset.classes}")

Detected classes: ['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato__Tomato_mosaic_virus', 'Tomato_healthy']


In [105]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [106]:
#Model Setup 

model = models.resnet18(pretrained=True)
for name, param in model.named_parameters():
        param.requires_grad = True
   
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

In [107]:
#Train the  model 
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [109]:
epochs = 15
for epoch in range(epochs): 
    running_loss = 0.0 
    correct = 0 
    total = 0 

    model.train() 

    for images, labels in train_loader:   
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()            
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()                  

        running_loss += loss.item()
        _, predicted = torch.max(output.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()


    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = 100 * correct / total
    print(f"Epoch [{epoch+1}/{epochs}] Loss: {epoch_loss:.4f} Accuracy: {epoch_acc:.2f}%")

    scheduler.step()                      


print(f"\nFinal Training Accuracy: {epoch_acc:.2f}%")


Epoch [1/15] Loss: 0.0172 Accuracy: 71.34%
Epoch [2/15] Loss: 0.0080 Accuracy: 86.27%
Epoch [3/15] Loss: 0.0063 Accuracy: 88.48%
Epoch [4/15] Loss: 0.0055 Accuracy: 89.57%
Epoch [5/15] Loss: 0.0050 Accuracy: 90.30%
Epoch [6/15] Loss: 0.0047 Accuracy: 91.19%
Epoch [7/15] Loss: 0.0046 Accuracy: 91.35%
Epoch [8/15] Loss: 0.0045 Accuracy: 91.71%
Epoch [9/15] Loss: 0.0046 Accuracy: 91.22%
Epoch [10/15] Loss: 0.0045 Accuracy: 91.41%
Epoch [11/15] Loss: 0.0045 Accuracy: 91.37%
Epoch [12/15] Loss: 0.0044 Accuracy: 91.62%
Epoch [13/15] Loss: 0.0044 Accuracy: 91.66%
Epoch [14/15] Loss: 0.0044 Accuracy: 91.62%
Epoch [15/15] Loss: 0.0043 Accuracy: 91.80%

Final Training Accuracy: 91.80%


**Checking how our model performed on test_data**

In [110]:
#Evaluate on Test Set
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_acc = 100 * correct / total
print(f"\n Test Accuracy: {test_acc:.2f}%")



 Test Accuracy: 92.64%


In [111]:
torch.save(model.state_dict(), "crop_disease_model.pth")
print("Model saved as crop_disease_model.pth")

Model saved as crop_disease_model.pth
